CONNECT TO DUCKDB TO PYTHON

In [4]:
import duckdb

with duckdb.connect("database/jokes.duckdb") as conn:
    conn.sql("""
        CREATE TABLE IF NOT EXISTS jokes (
            id INT,
            joke STRING
    );
""")

    conn.sql("""
        INSERT INTO jokes VALUES
            (1, 'Why do programmers hate sunlight, because they attract bugs'),
            (2, 'Why do java programmers need glasses, because they cannot C#');
    """)

    desc = conn.sql("DESC;").df()

